In [1]:
import os
import torch
import random
import datetime
import pandas as pd
import numpy as np

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, get_svd_encoder
from src.embeddings import RecsysEmbedding

experiment_name = 'InSlateAttention'
device = 'cuda:0'
seed = 123
pkl_path = '../data/'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
torch.__version__

'1.9.0'

# Модель

In [3]:
import torch.nn.functional as F
torch.autograd.set_detect_anomaly(True)

class AttentionResponseModel(torch.nn.Module):
    """
    No recurrent dependency, just slate-wise attention.
    """
    def __init__(self, embedding, nheads=2, output_dim=1):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.embedding = embedding
        self.attention= torch.nn.MultiheadAttention(
            2 * embedding.embedding_dim,
            num_heads=nheads,
            batch_first=True
        )
        
        self.out_layer = torch.nn.Linear(2 * embedding.embedding_dim, output_dim)

    def forward(self, batch):
        item_embs, user_embs = self.embedding(batch)
        shp = item_embs.shape
        
        # let model attent to first padd token if slate is empty to avoid NaN gradients
        # (anyway they does not contrinute into metrics computation)
        key_padding_mask = batch['slates_mask'].clone()
        key_padding_mask[:,:, 0] = True 
        
        # repeating user embedding for each item embeding
        features = torch.cat(
            [
                item_embs,
                user_embs[:, :, None, :].repeat(1, 1, shp[-2], 1).reshape(shp)
            ],
            dim = -1
        )
        
        # all recomendations in session are scored independently
        features = features.flatten(0,1)
        
        features, attn_map = self.attention(
            features, features, features,
            key_padding_mask=~key_padding_mask.flatten(0, 1)
        )
        
        # transforming back to sequence shape
        shp = list(shp)
        shp[-1] *= 2
        return self.out_layer(features.reshape(shp)).squeeze(-1)


    
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

for batch in dummy_loader:
    break

model = AttentionResponseModel(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural'),
    output_dim=1
).to('cpu')

model(batch)


3it [00:00, 2719.45it/s]

biulding affinity matrix...


tensor([[[ 0.0057,  0.0038,  0.0087],
         [ 0.0112,  0.0084,  0.0122]],

        [[-0.2585, -0.2585, -0.2585],
         [ 0.1548,  0.1548,  0.1548]]], grad_fn=<SqueezeBackward1>)

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [4]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

for embeddings in ['explicit', 'svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    model = AttentionResponseModel(
        RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
        output_dim=1
    ).to('cpu')
    _, metrics = train(model, 
       dummy_loader, dummy_loader, dummy_loader, device=device, lr=1e-3, num_epochs=5000, dummy=True,
       silent=True,
#        debug=True,
    
    )


3it [00:00, 3463.50it/s]

biulding affinity matrix...

Evaluating InSlateAttention with explicit embeddings



train... loss:0.6989719271659851:   0%|                                                                                      | 0/5000 [00:00<?, ?it/s]

Test before learning: {'f1': 0.0, 'roc-auc': 0.0, 'accuracy': 0.25}


train... loss:0.6672261357307434:   0%|                                                                              | 1/5000 [00:00<27:32,  3.03it/s]

...

Test update: epoch: 0 |accuracy: 0.25 | f1: 0.4000000059604645 | auc: 1.0 | treshold: 0.01


train... loss:0.6500577926635742:   0%|                                                                              | 2/5000 [00:00<26:16,  3.17it/s]

Test update: epoch: 1 |accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | treshold: 0.47000000000000003


train... loss:0.5874077081680298:   0%|                                                                              | 5/5000 [00:01<26:00,  3.20it/s]

Test update: epoch: 5 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.56


# ContentWise

In [5]:
content_wise_results = []
c = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
c_train_loader, c_val_loader, c_test_loader, c_train_user_item_matrix, train_num_items = get_train_val_test_tmatrix_tnumitems(c, batch_size=150)
len(c_train_loader), len(c)

(108, 20216)

In [6]:
for embeddings in ['neural', 'svd']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    model = AttentionResponseModel(
        RecsysEmbedding(train_num_items, c_train_user_item_matrix, embeddings='neural'),
        output_dim=1
    ).to(device)

    _, metrics = train(model, 
       c_train_loader, c_val_loader, c_test_loader, device=device, lr=1e-3, num_epochs=5000, early_stopping=10,
       silent=True,
#        debug=True
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)
    
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')


Evaluating InSlateAttention with neural embeddings


train:   0%|                                                                                                                 | 0/5000 [00:00<?, ?it/s]

Test before learning: {'f1': 0.1759941130876541, 'roc-auc': 0.5034826993942261, 'accuracy': 0.19906017184257507}


train... loss:45.46570086479187:   0%|                                                                            | 1/5000 [00:46<64:51:28, 46.71s/it]

Test update: epoch: 0 |accuracy: 0.75359046459198 | f1: 0.20581744611263275 | auc: 0.5939366817474365 | treshold: 0.11


...

train... loss:29.253538578748703:   0%|▏                                                                         | 13/5000 [07:27<48:31:52, 35.03s/it]

Test update: epoch: 12 |accuracy: 0.8259760141372681 | f1: 0.3224281966686249 | auc: 0.7458221912384033 | treshold: 0.18000000000000002


train... loss:28.245311856269836:   0%|▎                                                                         | 18/5000 [10:17<47:37:13, 34.41s/it]

Test update: epoch: 17 |accuracy: 0.8744612336158752 | f1: 0.31394559144973755 | auc: 0.7493888139724731 | treshold: 0.21000000000000002


train... loss:26.223515436053276:   1%|▉                                                                         | 63/5000 [37:24<48:51:21, 35.63s/it]


# RL4RS

In [7]:
rl4rs_results = []
r = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
r_train_loader, r_val_loader, r_test_loader, r_train_user_item_matrix, train_num_items = get_train_val_test_tmatrix_tnumitems(r, batch_size=20000)
len(r_train_loader), len(r)

(2, 45942)

In [8]:
for embeddings in ['neural','explicit', 'svd',  ]:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = AttentionResponseModel(
        RecsysEmbedding(train_num_items, r_train_user_item_matrix, embeddings='neural'),
        output_dim=1
    ).to(device)

    _, metrics = train(model, 
       r_train_loader, r_val_loader, r_test_loader, device=device, lr=1e-3, num_epochs=5000, early_stopping=50,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)
    
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')


Evaluating InSlateAttention with neural embeddings


train:   0%|                                                                                                                 | 0/5000 [00:00<?, ?it/s]

Test before learning: {'f1': 0.45464029908180237, 'roc-auc': 0.4483996629714966, 'accuracy': 0.4311933219432831}


train... loss:1.3914096355438232:   0%|                                                                           | 1/5000 [01:03<88:24:25, 63.67s/it]

Test update: epoch: 0 |accuracy: 0.6449522376060486 | f1: 0.7838669419288635 | auc: 0.5403372049331665 | treshold: 0.44


...

train... loss:0.7959638833999634:   8%|█████▋                                                                    | 383/5000 [35:57<7:32:57,  5.89s/it]

Test update: epoch: 382 |accuracy: 0.8479506969451904 | f1: 0.8842883706092834 | auc: 0.9256352186203003 | treshold: 0.5800000000000001


train... loss:0.8012086153030396:   8%|█████▋                                                                    | 384/5000 [36:03<7:31:25,  5.87s/it]

Test update: epoch: 383 |accuracy: 0.8475396037101746 | f1: 0.8844454884529114 | auc: 0.9256356954574585 | treshold: 0.55


train... loss:0.8092768788337708:   8%|█████▋                                                                    | 388/5000 [36:30<7:13:52,  5.64s/it]
